<a href="https://colab.research.google.com/github/ohudalraddadi-hub/ai-arabic-text-detection/blob/main/Task_2_1_preprocessing_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

file_path = "/content/drive/MyDrive/Colab Notebooks/arabic_generated_abstracts_long.csv"
df = pd.read_csv(file_path)
display(df.head())

,text,label,source_model,split_name
0,كثيرا ما ارتبطت المصادر التاريخية في الأندلس خ...,0,human,by_polishing
1,يعد العامل الثقافي احد ابرز الاسباب التي يعزى ...,0,human,by_polishing
2,شكلت تلك الجهود والمساعي الرائدة التي قام بها ...,0,human,by_polishing
3,يقوم المقال على اشكالية الضرائب الغير شرعية في...,0,human,by_polishing
4,تتفق المصادر التاريخية المتوفرة حول موضوع تطور...,0,human,by_polishing


In [ ]:
pip install pyarabic camel-tools nltk

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.4/126.4 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.7/124.7 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 89.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 96.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.4/608.4 kB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.3/122.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 94.2 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=03cd73e73db97c5a84b744d680a7941d487891e882715fe8d25c400e983311d1
  Stored in di

In [ ]:
!camel_data -i light

The following packages will be installed: 'morphology-db-msa-r13', 'morphology-db-egy-r13', 'disambig-mle-calima-msa-r13', 'dialectid-model26', 'morphology-db-glf-01', 'morphology-db-msa-s31', 'disambig-mle-calima-egy-r13', 'morphology-db-lev-01'
Extracting package 'morphology-db-msa-r13': 100% 40.5M/40.5M [00:00<00:00, 503MB/s]
Extracting package 'morphology-db-egy-r13': 100% 67.3M/67.3M [00:00<00:00, 303MB/s]
Extracting package 'disambig-mle-calima-msa-r13': 100% 88.7M/88.7M [00:01<00:00, 67.6MB/s]
Extracting package 'dialectid-model26': 100% 371M/371M [00:03<00:00, 93.3MB/s]
Extracting package 'morphology-db-glf-01': 100% 7.98M/7.98M [00:00<00:00, 171MB/s]
Extracting package 'morphology-db-msa-s31': 100% 44.8M/44.8M [00:00<00:00, 176MB/s]
Extracting package 'disambig-mle-calima-egy-r13': 100% 27.2M/27.2M [00:00<00:00, 163MB/s]
Extracting package 'morphology-db-lev-01': 100% 10.6M/10.6M [00:00<00:00, 197MB/s]


In [ ]:
from __future__ import annotations
import unicodedata
import regex as re
import pandas as pd

# --- CAMeL Tools: normalization, dediacritization, tokenization, disambiguation ---
from camel_tools.utils.normalize import (
    normalize_unicode,          # Unicode normalization
    normalize_alef_ar,          # أ/إ/آ/ٱ → ا
    normalize_teh_marbuta_ar,   # ة → ه
    normalize_alef_maksura_ar,  # ى → ي
)
from camel_tools.utils.dediac import dediac_ar         # remove tashkeel safely (keeps letters)
from camel_tools.tokenizers.word import simple_word_tokenize
from camel_tools.disambig.mle import MLEDisambiguator  # pretrained() MSA by default

# --- Stopwords ---
import nltk
from nltk.corpus import stopwords
try:
    _ = stopwords.words("arabic")
except LookupError:
    nltk.download("stopwords", quiet=True)

AR_STOPS_RAW = set(stopwords.words("arabic"))
# Normalize the stopword list with the SAME rules as text:
def _norm_for_stops(s: str) -> str:
    s = normalize_unicode(s, compatibility=True)   # NFC/NFKC
    s = dediac_ar(s)                               # strip diacritics
    s = normalize_alef_ar(s)                       # unify alef variants
    s = normalize_teh_marbuta_ar(s)                # ة→ه
    s = normalize_alef_maksura_ar(s)               # ى→ي
    s = re.sub(r"\s+", " ", s).strip()
    return s

AR_STOPS = {w for w in (_norm_for_stops(w) for w in AR_STOPS_RAW) if w}

# --- Build disambiguator (loads pretrained MSA model; requires camel_data installed) ---
# MLEDisambiguator.pretrained() uses default ‘calima-msa-r13’ unless specified.
mle = MLEDisambiguator.pretrained()

# --- Core normalization (Unicode → dediac → alef/teh/ى rules) ---
def normalize_arabic(s: str) -> str:
    if not isinstance(s, str):
        s = "" if s is None else str(s)
    s = normalize_unicode(s, compatibility=True)    # NFKC
    s = dediac_ar(s)                                # remove tashkeel (keeps letters)
    s = normalize_alef_ar(s)                        # أ/إ/آ/ٱ → ا
    s = normalize_teh_marbuta_ar(s)                 # ة → ه
    s = normalize_alef_maksura_ar(s)                # ى → ي
    # keep Arabic letters/digits/space only
    s = "".join(ch for ch in s if unicodedata.category(ch)[0] != "C")  # drop controls
    s = re.sub(r"[^\p{Arabic}\p{Nd}\s]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

# --- Tokenize with CAMeL’s whitespace+punct splitter ---
def tokenize_arabic(s: str) -> list[str]:
    if not s:
        return []
    # simple_word_tokenize is language-agnostic but respects Unicode punctuation boundaries
    return [t for t in simple_word_tokenize(s) if t.strip()]

# --- Lemmatize via morphological disambiguation (MSA) ---
def lemmas_via_camel(tokens: list[str]) -> list[str]:
    if not tokens:
        return []
    # Disambiguate expects a whitespace/punct-tokenized sentence
    disamb = mle.disambiguate(tokens)
    lemmas = []
    for d in disamb:
        # Take the top analysis when available; fall back to surface token
        if getattr(d, "analyses", None):
            # analyses[0].analysis is a dict; lemma is usually under 'lemma'
            lemmas.append(d.analyses[0].analysis.get("lemma", d.word))
        else:
            lemmas.append(d.word)
    return lemmas

# --- Full pipeline for one string ---
def preprocess_arabic(text: str, *, remove_stopwords: bool = True, to_lemmas: bool = True) -> str:
    s = normalize_arabic(text)
    toks = tokenize_arabic(s)
    if remove_stopwords:
        toks = [t for t in toks if t not in AR_STOPS]
    if to_lemmas:
        toks = lemmas_via_camel(toks)
        # if you removed stops before, consider removing again post-lemma
        if remove_stopwords:
            toks = [t for t in toks if t not in AR_STOPS]
    return " ".join(toks)

In [ ]:
df['text_clean'] = df['text'].apply(preprocess_arabic)

Frequently Word

In [ ]:
from collections import Counter
top = Counter(" ".join(df['text']).split()).most_common(50); top[:50]

[('في', 183151),
 ('على', 98459),
 ('من', 95287),
 ('إلى', 74853),
 ('هذه', 48616),
 ('الدراسة', 42331),
 ('البحث', 41323),
 ('أن', 35592),
 ('هذا', 32664),
 ('التي', 32412),
 ('بين', 29594),
 ('خلال', 26271),
 ('مع', 25326),
 ('عن', 16728),
 ('حيث', 16563),
 ('مما', 14262),
 ('تحليل', 13538),
 ('النتائج', 13313),
 ('تهدف', 13016),
 ('أهمية', 11137),
 ('دراسة', 11032),
 ('الذي', 10178),
 ('الضوء', 10006),
 ('التركيز', 9748),
 ('أو', 9543),
 ('كما', 9415),
 ('استخدام', 8877),
 ('فهم', 8824),
 ('يهدف', 8582),
 ('تم', 8557),
 ('تعزيز', 8556),
 ('دور', 8532),
 ('تأثير', 8421),
 ('و', 8162),
 ('الورقة', 7904),
 ('ما', 7873),
 ('بشكل', 7768),
 ('حول', 7652),
 ('ذلك', 7407),
 ('يمكن', 7201),
 ('اللغة', 7191),
 ('مثل', 7087),
 ('الجزائر', 6575),
 ('العربية', 6532),
 ('يتم', 6395),
 ('تحقيق', 6137),
 ('كيفية', 6114),
 ('لدى', 5840),
 ('هو', 5816),
 ('استكشاف', 5643)]

In [ ]:
from collections import Counter
top = Counter(" ".join(df['text_clean']).split()).most_common(50); top[:50]

[('الدراسه', 43979),
 ('البحث', 42597),
 ('خلال', 26298),
 ('النتائج', 13829),
 ('تحليل', 13567),
 ('تهدف', 13043),
 ('الجزائر', 12234),
 ('اهميه', 11243),
 ('دراسه', 11148),
 ('الضوء', 10033),
 ('العربيه', 9928),
 ('التركيز', 9785),
 ('الجزائري', 9326),
 ('استخدام', 8915),
 ('فهم', 8843),
 ('يهدف', 8656),
 ('دور', 8571),
 ('تعزيز', 8561),
 ('تاثير', 8437),
 ('اللغه', 8206),
 ('الورقه', 7956),
 ('بشكل', 7779),
 ('حول', 7668),
 ('يمكن', 7337),
 ('الاقتصاديه', 6958),
 ('الاجتماعيه', 6552),
 ('يتم', 6430),
 ('تحقيق', 6161),
 ('الجزائريه', 6135),
 ('كيفيه', 6128),
 ('القانونيه', 6126),
 ('العربي', 6035),
 ('النصوص', 5808),
 ('العلاقه', 5658),
 ('استكشاف', 5643),
 ('المجتمع', 5599),
 ('البحثيه', 5599),
 ('النص', 5234),
 ('المؤسسات', 5150),
 ('الاسلاميه', 5081),
 ('يتناول', 4997),
 ('لتحقيق', 4992),
 ('تحسين', 4861),
 ('بالاضافه', 4801),
 ('منهجيه', 4769),
 ('الماليه', 4665),
 ('تقديم', 4578),
 ('تشير', 4573),
 ('الاجتماعي', 4353),
 ('القانون', 4324)]

In [ ]:
cd /content/drive/MyDrive/Colab Notebooks

/content/drive/MyDrive/Colab Notebooks


In [ ]:
# Save per-split and combined parquet/csv
out_cols = ['text','text_clean3','label','source_model','split_name']
df[out_cols].to_parquet('arabic_generated_abstracts_long_clean2.parquet', index=False)
df[out_cols].to_csv('arabic_generated_abstracts_long_clean2.csv', index=False)
print("Saved: arabic_generated_abstracts_long_clean2.(parquet|csv)")

Saved: arabic_generated_abstracts_long_clean2.(parquet|csv)
